In [1]:
import os
import sys
import nltk
import csv
import re

In [2]:
with open('CSVs/tweet_data_20170825-155658.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    count = 0
    for row in spamreader:
        print(', '.join(row))
        count += 1
        if count > 100:
            break

901171567433990145, RayanZahab, 34.3833, 35.7833, https://t.co/0Qkpc4jkYh, Nowhere like home #northlebabon #livelovelebanon #kalamoun #port #beach #livelovenorth…
901171567429681152, smlifeislovely, 42.68034, -83.13422, https://t.co/QGvxm063ed, So fucking IN LOVE with my new pink hair!!! 💓💓💓 this makeup looks pretty 💣, too. #motd…
901171569719947266, sihlemakeup, -25.74641649, 28.1947812, https://t.co/nfhSCaDacH, Just posted a photo @ The South African State Theatre
901171574060965890, MichaelPlaster, 34.0522, -118.243, https://t.co/l7dElAnXK3, A part of the 24hr #Donation #Marathon to help #JavierVasquez beat down #ColonCancer //…
901171576418291714, CMMurrayLLP, 51.50514919, -0.07558474, https://t.co/kcrhSomaCf, Where it all began for our firm... #canarywharf framed by beautiful #towerbridge #11yearson @…
901171584269987840, DriiPPy, 51.56702967, 3.497227, https://t.co/LLVBtQShgI, D.R Do Whatever You Want...Because If You Just Listen To What Other Say You Will Never Do It And…
901171

In [3]:
posts = []

with open('CSVs/instagram_data_20170825-155658.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    count = 0
    for row in spamreader:
        text = row[2]
        posts.append(text)
        count += 1
        if count > 30:
            break

In [4]:
stopWords = set(nltk.corpus.stopwords.words('english'))
print(stopWords)

{'yourself', 's', 'our', 'between', 'about', 'too', 'so', 'above', 'am', 'all', 'such', 'after', 'to', 'couldn', 'himself', 'y', 'mustn', 'because', 'ourselves', 'was', 'themselves', 'aren', 'weren', 'both', 'being', 'o', 'than', 'further', 'doesn', 'those', 'before', 'an', 'more', 'at', 'she', 'in', 'hadn', 'out', 'other', 'there', 'does', 'its', 'once', 'ain', 'against', 'my', 'for', 'again', 'while', 'here', 'own', 'he', 'it', 'yourselves', 'theirs', 'nor', 'me', 'needn', 'when', 'we', 'were', 'her', 'mightn', 'had', 'you', 'most', 'and', 'their', 'his', 'this', 'over', 'haven', 'won', 'from', 'herself', 'by', 'very', 'should', 'ours', 'is', 'with', 'few', 'myself', 'not', 'how', 'into', 't', 'hasn', 'wouldn', 'been', 'shan', 'these', 'off', 'yours', 'as', 'of', 'if', 'him', 'didn', 'did', 'isn', 'll', 'shouldn', 'no', 'hers', 'i', 'doing', 'itself', 'through', 'what', 'don', 'they', 'have', 'having', 'be', 'some', 'do', 'up', 'will', 'has', 'ma', 'or', 'your', 'where', 'then', 're'

In [5]:
from nltk import TweetTokenizer
#tknz = TweetTokenizer(preserve_case=False, strip_handles=True)

for post in posts:
    print(post)
    # remove tagged users
    untagPost = re.sub('@[\w.-]+', '', post)
    
    # tokenize and remove stopwords
    words = []
    for word in nltk.word_tokenize(untagPost):
        if word not in stopWords and re.match('^[A-Za-z]+$', word) and len(word) > 2:
            words.append(word.lower())
    print(words)
    print()
    #print(tknz.tokenize(post))
    

Nowhere like home #northlebabon #livelovelebanon #kalamoun #port #beach #livelovenorth #lebanesenature #middeterranean
['nowhere', 'like', 'home', 'northlebabon', 'livelovelebanon', 'kalamoun', 'port', 'beach', 'livelovenorth', 'lebanesenature', 'middeterranean']

So fucking IN LOVE with my new pink hair!!! 💓💓💓 this makeup looks pretty 💣, too. #motd #makeupaddict #aspiringmua #pinkhair #smashbox #smashboxprimer #maybelline #maccosmetics #macpaintpot #macfixplus #tarte #tarteshapetape #tarteamazonianclayblush #jouer #jouerhighlightonfleek #benefit #benefitrollerlash #benefithoolabronzer #karity #karitymattepalette #morphebrushes #morphebabe #morphepalette #morpheliquidlipstick #anastasiabeverlyhills #anastasiabrowwiz #toofaced #toofacedquickiebrow #toofacedhangover
['fucking', 'love', 'new', 'pink', 'hair', 'makeup', 'looks', 'pretty', 'motd', 'makeupaddict', 'aspiringmua', 'pinkhair', 'smashbox', 'smashboxprimer', 'maybelline', 'maccosmetics', 'macpaintpot', 'macfixplus', 'tarte', 'tar